## Import thư viện

In [1]:
# Some part of the code was referenced from below.
# https://github.com/pytorch/examples/tree/master/word_language_model 
import os
import torch
import torch.nn as nn
import numpy as np
from torch.nn.utils import clip_grad_norm_

## Các lớp helper

In [2]:
class Dictionary(object):
    def __init__(self):
        self.word2idx = {}
        self.idx2word = {}
        self.idx = 0
    # Phương thức thêm từ
    def add_word(self, word):
        if not word in self.word2idx:
            self.word2idx[word] = self.idx
            self.idx2word[self.idx] = word
            self.idx += 1
    # Phương thức lấy kích thước
    def __len__(self):
        return len(self.word2idx)

In [3]:
class Corpus(object):
    def __init__(self):
        self.dictionary = Dictionary()

    def get_data(self, path, batch_size=20):
        # Thêm từ vào từ điển
        with open(path, 'r') as f:
            tokens = 0
            for line in f:
                words = line.split() + ['<eos>']
                tokens += len(words)
                for word in words: 
                    self.dictionary.add_word(word)  
        
        # Tokenize nội dung tập tin
        ids = torch.LongTensor(tokens)
        token = 0
        with open(path, 'r') as f:
            for line in f:
                words = line.split() + ['<eos>']
                for word in words:
                    ids[token] = self.dictionary.word2idx[word]
                    token += 1
        num_batches = ids.size(0) // batch_size
        ids = ids[:num_batches*batch_size]
        return ids.view(batch_size, -1)

## Thiết lập device

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

## Đặt các siêu tham số (Hyper-parameters)

In [5]:
# Kích thước embed
embed_size = 128

# Kích thước ẩn
hidden_size = 1024

# Số tầng
num_layers = 1

# Số lượng epochs
num_epochs = 5

num_samples = 1000     # number of words to be sampled

# Kích thước mỗi batch
batch_size = 20

# Chiều dài chuỗi
seq_length = 30

# Tốc độ học
learning_rate = 0.002

## Tải dữ liệu "Penn Treebank" dataset

In [6]:
corpus = Corpus()
ids = corpus.get_data('data/train.txt', batch_size)
vocab_size = len(corpus.dictionary)
num_batches = ids.size(1) // seq_length

## Mô hình RNN dựa trên mô hình ngôn ngữ (RNN based language model)

In [7]:
class RNNLM(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers):
        super(RNNLM, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_size, vocab_size)
        
    def forward(self, x, h):
        # Embed word ids thành vectors
        x = self.embed(x)
        
        # Forward propagate LSTM
        out, (h, c) = self.lstm(x, h)
        
        # Reshape output to (batch_size*sequence_length, hidden_size)
        out = out.reshape(out.size(0)*out.size(1), out.size(2))
        
        # Decode hidden states of all time steps
        out = self.linear(out)
        return out, (h, c)

In [8]:
model = RNNLM(vocab_size, embed_size, hidden_size, num_layers).to(device)
model

RNNLM(
  (embed): Embedding(10000, 128)
  (lstm): LSTM(128, 1024, batch_first=True)
  (linear): Linear(in_features=1024, out_features=10000, bias=True)
)

## Hàm mất mát (loss function) và bộ tối ưu hóa (optimizer)

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

## Cắt ngắn lan truyền (Truncated backpropagation)

In [10]:
def detach(states):
    return [state.detach() for state in states] 

## Huấn luyện mô hình

In [11]:
for epoch in range(num_epochs):
    # Set initial hidden and cell states
    states = (torch.zeros(num_layers, batch_size, hidden_size).to(device),
              torch.zeros(num_layers, batch_size, hidden_size).to(device))
    
    for i in range(0, ids.size(1) - seq_length, seq_length):
        # Get mini-batch inputs and targets
        inputs = ids[:, i:i+seq_length].to(device)
        targets = ids[:, (i+1):(i+1)+seq_length].to(device)
        
        # Forward pass
        states = detach(states)
        outputs, states = model(inputs, states)
        loss = criterion(outputs, targets.reshape(-1))
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        step = (i+1) // seq_length
        if step % 100 == 0:
            print ('Epoch [{}/{}], Step[{}/{}], Loss: {:.4f}, Perplexity: {:5.2f}'
                   .format(epoch+1, num_epochs, step, num_batches, loss.item(), np.exp(loss.item())))

Epoch [1/5], Step[0/1549], Loss: 9.2138, Perplexity: 10034.35
Epoch [1/5], Step[100/1549], Loss: 6.0557, Perplexity: 426.54
Epoch [1/5], Step[200/1549], Loss: 5.9174, Perplexity: 371.46
Epoch [1/5], Step[300/1549], Loss: 5.7649, Perplexity: 318.90
Epoch [1/5], Step[400/1549], Loss: 5.6913, Perplexity: 296.27
Epoch [1/5], Step[500/1549], Loss: 5.0923, Perplexity: 162.76
Epoch [1/5], Step[600/1549], Loss: 5.1487, Perplexity: 172.21
Epoch [1/5], Step[700/1549], Loss: 5.3450, Perplexity: 209.55
Epoch [1/5], Step[800/1549], Loss: 5.1938, Perplexity: 180.15
Epoch [1/5], Step[900/1549], Loss: 5.0499, Perplexity: 156.00
Epoch [1/5], Step[1000/1549], Loss: 5.0828, Perplexity: 161.23
Epoch [1/5], Step[1100/1549], Loss: 5.3525, Perplexity: 211.14
Epoch [1/5], Step[1200/1549], Loss: 5.1564, Perplexity: 173.53
Epoch [1/5], Step[1300/1549], Loss: 5.0545, Perplexity: 156.73
Epoch [1/5], Step[1400/1549], Loss: 4.8150, Perplexity: 123.35
Epoch [1/5], Step[1500/1549], Loss: 5.1747, Perplexity: 176.75
Ep

## Kiểm tra mô hình

In [12]:
with torch.no_grad():
    with open('sample.txt', 'w') as f:
        # Set intial hidden ane cell states
        # Khởi tạo những trạng thái ẩn và cell
        state = (torch.zeros(num_layers, 1, hidden_size).to(device),
                 torch.zeros(num_layers, 1, hidden_size).to(device))

        # Chọn một word id ngẫu nhiên
        prob = torch.ones(vocab_size)
        input = torch.multinomial(prob, num_samples=1).unsqueeze(1).to(device)

        for i in range(num_samples):
            # Forward propagate RNN 
            output, state = model(input, state)

            # Sample a word id
            prob = output.exp()
            word_id = torch.multinomial(prob, num_samples=1).item()

            # Fill input with sampled word id for the next time step
            input.fill_(word_id)

            # File write
            word = corpus.dictionary.idx2word[word_id]
            word = '\n' if word == '<eos>' else word + ' '
            f.write(word)

            if (i+1) % 100 == 0:
                print('Sampled [{}/{}] words and save to {}'.format(i+1, num_samples, 'sample.txt'))

Sampled [100/1000] words and save to sample.txt
Sampled [200/1000] words and save to sample.txt
Sampled [300/1000] words and save to sample.txt
Sampled [400/1000] words and save to sample.txt
Sampled [500/1000] words and save to sample.txt
Sampled [600/1000] words and save to sample.txt
Sampled [700/1000] words and save to sample.txt
Sampled [800/1000] words and save to sample.txt
Sampled [900/1000] words and save to sample.txt
Sampled [1000/1000] words and save to sample.txt


## Lưu mô hình (model checkpoint)

In [13]:
torch.save(model.state_dict(), 'model.ckpt')